In [1]:
from dotenv import load_dotenv
load_dotenv()
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to("cuda" if torch.cuda.is_available() else "cpu")


/Users/alfio/python_venv/general/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dags.lib.PineconeManager import PineconeManager
import os
pinecone_key = os.getenv("PINECONE_API")
index_name = "letstalkvector"
namespace = "letstalk-ns"

pi = PineconeManager(index_name, namespace, pinecone_key)

In [3]:
interest = "Apple"

response = pi.query(interest, num_results=1)

In [7]:
result = response.get("result").get('hits')[0].get('fields').get('text_to_embed')

'apple  aapl  usually can count on a positive news cycle around its annual worldwide developers conference  but expectations this year are subdued  and that sentiment is reflected in apple stock actio         chars'

In [64]:
input_text = f"""Generate exactly ONE question about the following topic. Do NOT answer it. Make it FUN and INFORMAL. Do not refer directly to what I'm passing you as the user can't see it.
Topic: {result}
Question:"""

inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=40,
    do_sample=True,
    temperature=0.7,
    top_k=40,
    top_p=0.9,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.2
)

for i, out in enumerate(outputs):
    full_text = tokenizer.decode(out, skip_special_tokens=True)
    question = full_text.split("Question:")[-1].split("\n")[0].split("?")[0].strip()
    print(f"Option {i+1}: {question}?")

Option 1: Can you summarize the current sentiment towards Apple ahead of their Worldwide Developers Conference?
Option 2: Can you summarize the article about Apple's upcoming Worldwide Developers Conference?
Option 3: How did Apple's Worldwide Developers Conference (WWDC) perform during last year?


In [65]:
response

{'result': {'hits': [{'_id': 'https://www.investors.com/news/technology/apple-stock-wwdc-2025-expectations-low/',
                      '_score': 0.34259694814682007,
                      'fields': {'author': "PATRICK SEITZ, Investor's Business "
                                           'Daily',
                                 'content': 'apple  aapl  usually can count on '
                                            'a positive news cycle around its '
                                            'annual worldwide developers '
                                            'conference  but expectations this '
                                            'year are subdued  and that '
                                            'sentiment is reflected in apple '
                                            'stock actio         chars',
                                 'data_source': 'delta_news/technology',
                                 'description': 'expectations for apple s '
        